# Find KEGG associations

This notebook will create a table that has the KEGG pathways that are associated with all genes, but we are particularly interested in those that are associated with the most and least stable genes.

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import os
import random
import pandas as pd
from scripts import paths, utils, modules, annotations

random.seed(1)

In [2]:
# Output files
pao1_out_filename = "pao1_core_similarity_associations_spell.tsv"
pa14_out_filename = "pa14_core_similarity_associations_spell.tsv"

In [3]:
# Load transcriptional similarity df
pao1_similarity_scores_filename = "pao1_core_similarity_expression_stats_spell.tsv"
pa14_similarity_scores_filename = "pa14_core_similarity_expression_stats_spell.tsv"

pao1_similarity_scores = pd.read_csv(
    pao1_similarity_scores_filename, sep="\t", header=0, index_col=0
)
pa14_similarity_scores = pd.read_csv(
    pa14_similarity_scores_filename, sep="\t", header=0, index_col=0
)

In [4]:
print(pao1_similarity_scores.shape)
pao1_similarity_scores.head()

(5349, 14)


,PA14 homolog id,Transcriptional similarity across strains,P-value,Name,label,mean expression,standard deviation expression,min expression,25% expression,50% expression,75% expression,max expression,variance expression,range expression
PAO1 id,,,,,,,,,,,,,,
PA0118,PA14_01440,0.290443,1.771504e-104,NaN,NaN,84.379666,87.012773,0.000000,30.805250,56.083433,98.678794,743.594397,7.571223e+03,743.594397
PA1859,PA14_40440,0.143328,5.986934e-26,NaN,NaN,106.687908,91.482549,0.000000,50.280627,78.021322,118.345717,584.117921,8.369057e+03,584.117921
PA3190,PA14_22980,0.257508,9.170620e-82,NaN,NaN,2974.728597,4619.591491,0.000000,477.708748,1386.138060,3291.525419,41815.193043,2.134063e+07,41815.193043
PA1009,PA14_51280,0.549950,0.000000e+00,NaN,NaN,448.628362,387.755818,18.128903,238.075927,370.338444,543.930949,5826.229102,1.503546e+05,5808.100199
PA1065,PA14_50620,0.329633,9.188709e-136,NaN,NaN,105.604816,85.534956,0.000000,47.418993,82.907779,144.322724,707.184303,7.316229e+03,707.184303


In [5]:
print(pa14_similarity_scores.shape)
pa14_similarity_scores.head()

(5347, 14)


,PAO1 homolog id,Transcriptional similarity across strains,P-value,Name,label,mean expression,standard deviation expression,min expression,25% expression,50% expression,75% expression,max expression,variance expression,range expression
PA14 id,,,,,,,,,,,,,,
PA14_23690,PA3125,0.185094,2.030432e-42,NaN,NaN,85.595452,101.121951,0.000000,25.546082,51.093537,104.536787,709.117909,1.022565e+04,709.117909
PA14_11480,PA4049,0.327795,3.869112e-134,NaN,NaN,198.132451,121.601925,0.000000,125.119670,168.451503,230.506602,1084.278966,1.478703e+04,1084.278966
PA14_01150,PA0094,0.418704,4.989573e-226,NaN,NaN,105.290577,111.218985,0.000000,39.297006,75.807135,124.150007,860.205738,1.236966e+04,860.205738
PA14_11460,PA4051,0.541759,0.000000e+00,thiL,NaN,234.395250,177.782645,16.650153,110.935215,188.025557,300.223243,1008.676242,3.160667e+04,992.026089
PA14_56780,PA4366,0.482227,1.451201e-309,sodB,NaN,7781.835203,7633.163594,412.593023,3450.004296,5718.029115,8591.327252,72482.666616,5.826519e+07,72070.073593


In [6]:
# Load KEGG pathway data
pao1_pathway_filename = "https://raw.githubusercontent.com/greenelab/adage/7a4eda39d360b224268921dc1f2c14b32788ab16/Node_interpretation/pseudomonas_KEGG_terms.txt"

In [7]:
pao1_pathways = annotations.load_format_KEGG(pao1_pathway_filename)
pao1_pathways.head()

,1,2
0,,
KEGG-Pathway-pae00072: Synthesis and degradation of ketone bodies,10,"{PA2011, PA2553, PA1736, PA2003, PA2000, PA200..."
KEGG-Pathway-pae00071: Fatty acid degradation,32,"{PA3014, PA2553, PA2940, PA2550, PA1737, PA200..."
KEGG-Pathway-pae00903: Limonene and pinene degradation,9,"{PA1821, PA3014, PA2475, PA3331, PA1737, PA102..."
KEGG-Pathway-pae00380: Tryptophan metabolism,27,"{PA3014, PA2553, PA3366, PA2579, PA0202, PA173..."
KEGG-Pathway-pae00900: Terpenoid backbone biosynthesis,16,"{PA2553, PA1736, PA3650, PA4569, PA4043, PA200..."


## Pathway annotations to PA14

The annotations we have are only for PAO1 genes, so we will map PAO1 core genes to PA14 core genes to add annotations to PA14. This is possible since we are focused on only core genes, which have homologs between PAO1 and PA14

In [8]:
pao1_annotation_filename = paths.GENE_PAO1_ANNOT
gene_mapping_pao1 = utils.get_pao1_pa14_gene_map(pao1_annotation_filename, "pao1")

In [9]:
gene_mapping_pao1 = gene_mapping_pao1["PA14_ID"].to_frame()

## Get pathway associations for all genes

In [10]:
def get_associated_pathways(genes_):
    rows = []
    for gene_id in genes_:
        pathway_bool = [
            gene_id in pao1_pathways.loc[pathway, 2] for pathway in pao1_pathways.index
        ]
        found_pathways = list(pao1_pathways[pathway_bool].index)
        rows.append({"gene id": gene_id, "pathways present": found_pathways})
    return pd.DataFrame(rows).set_index("gene id")

In [11]:
# Get KEGG associations for all genes in PAO1
all_pao1_gene_ids = list(pao1_similarity_scores.index)
pao1_associations = get_associated_pathways(all_pao1_gene_ids)

In [12]:
print(pao1_associations.shape)
pao1_associations.head()

(5349, 1)


,pathways present
gene id,
PA0118,[]
PA1859,[]
PA3190,[]
PA1009,[]
PA1065,[]


In [13]:
# Map PA14 gene ids
pa14_associations = pao1_associations.merge(
    gene_mapping_pao1, left_index=True, right_index=True
)
pa14_associations.set_index("PA14_ID", inplace=True)
print(pa14_associations.shape)
pa14_associations.head()

(5349, 1)


,pathways present
PA14_ID,
PA14_01440,[]
PA14_40440,[]
PA14_22980,[]
PA14_51280,[]
PA14_50620,[]


In [14]:
# Merge KEGG associations with transcriptional similarity information
pao1_associations = pao1_similarity_scores.merge(
    pao1_associations, left_index=True, right_index=True, how="left"
)
pa14_associations = pa14_similarity_scores.merge(
    pa14_associations, left_index=True, right_index=True, how="left"
)

In [15]:
print(pao1_associations.shape)
pao1_associations.head()

(5349, 15)


,PA14 homolog id,Transcriptional similarity across strains,P-value,Name,label,mean expression,standard deviation expression,min expression,25% expression,50% expression,75% expression,max expression,variance expression,range expression,pathways present
PAO1 id,,,,,,,,,,,,,,,
PA0118,PA14_01440,0.290443,1.771504e-104,NaN,NaN,84.379666,87.012773,0.000000,30.805250,56.083433,98.678794,743.594397,7.571223e+03,743.594397,[]
PA1859,PA14_40440,0.143328,5.986934e-26,NaN,NaN,106.687908,91.482549,0.000000,50.280627,78.021322,118.345717,584.117921,8.369057e+03,584.117921,[]
PA3190,PA14_22980,0.257508,9.170620e-82,NaN,NaN,2974.728597,4619.591491,0.000000,477.708748,1386.138060,3291.525419,41815.193043,2.134063e+07,41815.193043,[]
PA1009,PA14_51280,0.549950,0.000000e+00,NaN,NaN,448.628362,387.755818,18.128903,238.075927,370.338444,543.930949,5826.229102,1.503546e+05,5808.100199,[]
PA1065,PA14_50620,0.329633,9.188709e-136,NaN,NaN,105.604816,85.534956,0.000000,47.418993,82.907779,144.322724,707.184303,7.316229e+03,707.184303,[]


In [16]:
print(pa14_associations.shape)
pa14_associations.head()

(5347, 15)


,PAO1 homolog id,Transcriptional similarity across strains,P-value,Name,label,mean expression,standard deviation expression,min expression,25% expression,50% expression,75% expression,max expression,variance expression,range expression,pathways present
PA14 id,,,,,,,,,,,,,,,
PA14_23690,PA3125,0.185094,2.030432e-42,NaN,NaN,85.595452,101.121951,0.000000,25.546082,51.093537,104.536787,709.117909,1.022565e+04,709.117909,[]
PA14_11480,PA4049,0.327795,3.869112e-134,NaN,NaN,198.132451,121.601925,0.000000,125.119670,168.451503,230.506602,1084.278966,1.478703e+04,1084.278966,[]
PA14_01150,PA0094,0.418704,4.989573e-226,NaN,NaN,105.290577,111.218985,0.000000,39.297006,75.807135,124.150007,860.205738,1.236966e+04,860.205738,[]
PA14_11460,PA4051,0.541759,0.000000e+00,thiL,NaN,234.395250,177.782645,16.650153,110.935215,188.025557,300.223243,1008.676242,3.160667e+04,992.026089,[KEGG-Pathway-pae00730: Thiamine metabolism]
PA14_56780,PA4366,0.482227,1.451201e-309,sodB,NaN,7781.835203,7633.163594,412.593023,3450.004296,5718.029115,8591.327252,72482.666616,5.826519e+07,72070.073593,[]


In [17]:
# Save
pao1_associations.to_csv(pao1_out_filename, sep="\t")
pa14_associations.to_csv(pa14_out_filename, sep="\t")